'''
[KSensor] Data checker daily.ipynb
author: Geonwoo Ji
modifier: Geonwoo Ji
ver 1.0

PM data checker
'''

In [9]:
# This libraries will use for multi-processing
# from concurrent.futures import thread
# from distutils import core

import os
import math
import time
import json
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [10]:
IoT_1 = 'http://114.71.220.59:7579'
MOBIUS = '/Mobius/'
aeName = 'gpm_ubicomp/'
container = 'final_data'

resArc = IoT_1 + MOBIUS + aeName + container

payload={}
headers = {
    'Accept': 'application/json',
    'X-M2M-RI': '12345',
    'X-M2M-Origin': 'ubicomp_super'
}

In [11]:
def monitorPrg(date):
    # date = '20231024'
    date = str('20'+date)
    cra = datetime.strptime(date, "%Y%m%d")
    cra = cra - timedelta(1)
    cra = '&cra=' + cra.strftime("%Y%m%d") + 'T150000'
    crb = '&crb=' + date + 'T145959'

    url1 = resArc + "?fu=2&ty=4&rcn=4" + cra + crb

    try:
        dataRes = json.loads(requests.request("GET", url1, headers=headers, data=payload).text)
        dataRes = dataRes['m2m:rsp']['m2m:cin']
        gpm_cin = len(dataRes)
        extData = [{'ct': item['ct'], 'con': item['con']} for item in dataRes]
        buf_lst = []
        for buf in extData:
            splitCon = buf['con'].split(',')
            ext = {
                # 'createTime': buf['ct'],
                'datetime': int(splitCon[1]),
                'id': splitCon[0],
                'pm1': splitCon[2],
                'pm2.5': splitCon[3],
                'pm10': splitCon[4],
                'CO': splitCon[5],
                'NO2': splitCon[6],
                'SO2': splitCon[7],
                'O3': splitCon[8],
                'humidity':splitCon[9],
                'temperature' : splitCon[10],
                'voltage':splitCon[11]
            }
            buf_lst.append(ext)
        df = pd.DataFrame(buf_lst)
        df = df.sort_values(by=['datetime','id'])
        df = df.reset_index(drop=True)

        return df
    except Exception as e:
        print("Error occurred in sector 1", e)

In [12]:
# def queryCount(date, startT, endT):
#     df = monitorPrg(date)
#     qur = df[(df['datetime'] >= int(date + startT)) & (df['datetime'] <= int(date + endT))]
#     print(qur.count())
#     print(qur)

# queryCount("231024", "100000", "150000")

In [13]:
date = '231129'
startT = '000000'
endT = '240000'

df = monitorPrg(date)
qur = df[(df['datetime'] >= int(date + startT)) & (df['datetime'] <= int(date + endT))]
qur = qur.reset_index(drop=True)
print('count CIN:', qur['id'].count())
qur
#df

count CIN: 152


,datetime,id,pm1,pm2.5,pm10,CO,NO2,SO2,O3,humidity,temperature,voltage
0,231129204001,1,22,34,36,1.819,0.036,0.0,0.02,66.40,1.40,4.11
1,231129204001,2,23,35,39,1.442,0.0,0.0,0.02,65.00,0.60,4.09
2,231129204001,3,26,40,51,1.462,0.014,0.0,0.11,66.10,0.20,4.17
3,231129204001,4,27,39,47,1.762,0.0,0.002,0.056,65.60,0.70,4.22
4,231129204501,1,24,36,47,1.803,0.034,0.0,0.02,65.40,1.30,4.10
...,...,...,...,...,...,...,...,...,...,...,...,...
147,231129234501,2,4,7,7,0.976,0.0,0.0,0.02,58.30,-0.60,4.13
148,231129234501,4,8,9,11,1.317,0.0,0.0,0.047,57.00,-0.70,4.16
149,231129235001,1,5,6,6,1.441,0.025,0.0,0.02,52.90,0.30,4.09
150,231129235001,2,2,4,8,0.978,0.0,0.0,0.02,57.80,-0.70,4.12
